In [32]:
import pandas as pd

data_raw = pd.read_excel('.\\resources\\datamined_data_raw\\armor.xls')
data_raw
print(len(data_raw))
data_raw.columns




3480


Index(['Name', 'Index', 'Order', 'Variant', 'Set (Layered) Id', 'Type',
       'Equip Slot', 'Defense', 'Model Id 1', 'Model Id 2', 'Icon color',
       'Icon Effect', 'Rarity', 'Cost', 'Fire Res', 'Water Res', 'Ice Res',
       'Thuder Res', 'Dragon Res', 'Slot Count', 'Slot 1 Size', 'Slot 2 Size',
       'Slot 3 Size', 'Set Skill 1', 'Set Skill 1 Level', 'Hidden Skill',
       'Hidden Skill Level', 'Skill 1', 'Skill 1 Level', 'Skill 2',
       'Skill 2 Level', 'Skill 3', 'Skill 3 Level', 'Gender', 'Set Group',
       'Is_Permanent', 'Description'],
      dtype='object')

### Remove Layered Armor

In [33]:
data_raw['Type'].unique()

# Remove dummy/filler rows
data = data_raw[(data_raw['Type'] == 'Regular') | (data_raw['Type'] == 'Full_Set')]



### Identify Duplicate Names, Descriptions and remove bad values

In [34]:
byName = data.groupby('Name').size()
byName[byName > 1]
# Note the Butterfly Brachia pieces showing up as duplicate is intended; Male/Female sets

data = data[~data['Name'].isin(['Unavailable', 'HARDUMMY', 'Invalid Message'])]
data = data[~((data['Name'] == 'Leather Headgear') & (data['Description'] == 'Leather Headgear')) ]
data = data[~(data['Description'] == "Unavailable")]

In [35]:
data['Equip Slot'].unique()


array(['Head', 'Chest', 'Arms', 'Waist', 'Legs', 'Charm'], dtype=object)

In [36]:

for col in data:
  print([col, len(data[col].unique())])
data['Set Skill 1'].unique()

['Name', 1990]
['Index', 2005]
['Order', 484]
['Variant', 3]
['Set (Layered) Id', 374]
['Type', 2]
['Equip Slot', 6]
['Defense', 62]
['Model Id 1', 116]
['Model Id 2', 13]
['Icon color', 9]
['Icon Effect', 8]
['Rarity', 12]
['Cost', 22]
['Fire Res', 9]
['Water Res', 10]
['Ice Res', 11]
['Thuder Res', 11]
['Dragon Res', 10]
['Slot Count', 4]
['Slot 1 Size', 5]
['Slot 2 Size', 5]
['Slot 3 Size', 5]
['Set Skill 1', 72]
['Set Skill 1 Level', 2]
['Hidden Skill', 1]
['Hidden Skill Level', 1]
['Skill 1', 122]
['Skill 1 Level', 6]
['Skill 2', 108]
['Skill 2 Level', 5]
['Skill 3', 1]
['Skill 3 Level', 1]
['Gender', 3]
['Set Group', 455]
['Is_Permanent', 2]
['Description', 604]


array(['0: --------', '125: Anjanath Power', '151: Legiana Blessing',
       '153: Odogaron Power', '128: Rathalos Power', '129: Diablos Power',
       '130: Kirin Blessing', '132: Anjanath Will',
       '131: Pink Rathian Mastery', '152: Legiana Favor',
       '154: Odogaron Mastery', '135: Rathalos Mastery',
       '136: Diablos Mastery', '138: Uragaan Protection',
       '126: Nergigante Hunger', '144: Kushala Daora Flight',
       '143: Teostra Technique', '137: Kirin Favor',
       '127: Vaal Hazak Vitality', '142: Bazelgeuse Protection',
       '149: Guild Guidance', "145: Xeno'jiiva Divinity",
       '147: Zorah Magdaros Mastery', '150: Commission Guidance',
       '168: Astera Blessing', '164: Soul of the Dragoon',
       '163: Lunastra Favor', "167: Witcher's Knowledge",
       '182: Anjanath Dominance', '183: Rathian Essence',
       '188: Barioth Hidden Art', '189: Rathalos Essence',
       '190: Diablos Ambition', '191: Legiana Ambition',
       '192: Odogaron Essence', '19

### Investigating and fixing Set Bonuses. There must have been a few offset data in here, there were a bunch of nonsensical sets here that need manual patching

In [37]:
data_cp = data
data_cp_armor_only = data_cp
data_cp_armor_only = data_cp_armor_only[~(data_cp_armor_only['Equip Slot'] == 'Charm')]
set_group_by_size = data_cp_armor_only.groupby('Set Group').agg(set_group_size = ('Set Group', 'size'))
set_group_by_size.sort_values(by='set_group_size')
set_group_by_size.groupby('set_group_size').size()
data_cp_armor_only = data_cp_armor_only.merge(set_group_by_size, on='Set Group')
data_cp_armor_only

data_cp_armor_only[((data_cp_armor_only['set_group_size'] == 3))].sort_values(by='Name')
# data_cp_armor_only[data_cp_armor_only['Name'].str.contains("Butterfly")]
# data_cp_armor_only[data_cp_armor_only['Set Group'].isin([248, 250, 259])]
# data_raw[data_raw['Name'].str.contains("Kushala")]
# data_cp_armor_only.sort_values(by=['Set Group', 'Equip Slot']).to_csv("sorted_armor.csv")

#data[data['Order'] == 0]

group_by_order = data.groupby('Order').size().to_frame('Size').sort_values(['Size','Order'])
group_by_order

data[data['Order'] == 2]

# Fix Set Group data
equivalent_set_groups = [
  [159, 169], # Butterfly Female LR
  [160, 170], # Butterfly Female HR A
  [161, 171], # Butterfly Female HR B
  [149, 151], # Commission Alpha
  [150, 152], # Commission Beta
  [164, 167, 176], # Origin*
  [242, 244, 253], # Empress Gamma
  [247, 249, 258], # Kulve Taroth Gamma
  [248, 250, 259], # Defender
  ] #Buff, Butterfly, Commission, Kushala, Origin, Ryu

for pieces in equivalent_set_groups:
  # rowsToUpdate = data[(data['Set Group'].isin(pieces) & ~(data['Equip Slot'] == 'Charm'))]
  # print(rowsToUpdate['Name'])
  # print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  data.loc[(data['Set Group'].isin(pieces) & ~(data['Equip Slot'] == 'Charm')), 'Set Group'] = min(pieces)

# for pieces in equivalent_set_groups:
#   print(data[data['Set Group'] == min(pieces)]['Name'])

singleton_pieces_in_groups = [
    'Gastodon Horn α',
    'Gastodon Horn β',
    'Kestodon Guards β',
    'Skull Mask α',
    'Sealed Eyepatch α',
    'Shadow Shades α',
    'Kestodon Guards'
  ]

set_groups = [1001, 1002, 1003, 1004, 1005, 1006, 1007]

data.loc[(data['Name'].isin(singleton_pieces_in_groups)), 'Set Group'] = set_groups


shifted_armor_pieces = [
  # Piece / Should Be
  ('Kushala Cocoon γ', 221), # +1
  ('Nergigante Coil γ', 222),
  ('Zorah Spine γ', 223),
  ('Xeno\'jiiva Spine γ', 224),
  ('Ryu\'s Torso', 226), # +2
  ('Ryu\'s Torso α', 227),
  ('Sakura\'s Skirt α', 228),
  ('Azure Starlord Tassets α', 229),
  ('Dante\'s Belt α', 230),
  ('Empress Coil α', 231),
  ('Empress Coil β', 232),
  ]

for (name, newSetGroup) in shifted_armor_pieces:
  data.loc[(data['Name'] == name), 'Set Group'] = newSetGroup



### Exploring Descriptions

In [38]:
data_by_desc = data_cp_armor_only
# data_desc_count = data_by_desc.groupby(by='Description').agg(descSize = ('Description', 'size'))
# data_by_desc_j = data_by_desc.merge(data_desc_count,on='Description')
# data_by_desc_j.groupby(by='descSize').size()
#data_by_desc_j[data_by_desc_j['descSize'] == 3].sort_values('Description')



# print(len(data))
# data.columns

#### Review Shifted Armor Sets

In [39]:
data_cp = data
data_cp_armor_only = data_cp
data_cp_armor_only = data_cp_armor_only[~(data_cp_armor_only['Equip Slot'] == 'Charm')]
set_group_by_size = data_cp_armor_only.groupby('Set Group').agg(set_group_size = ('Set Group', 'size'))
set_group_by_size.sort_values(by='set_group_size')
set_group_by_size.groupby('set_group_size').size()
# data_cp_armor_only = data_cp_armor_only.merge(set_group_by_size, on='Set Group')
# data_cp_armor_only

# data_cp_armor_only[((data_cp_armor_only['set_group_size'] > 5))].sort_values(by='Name')
# # data_cp_armor_only[data_cp_armor_only['set_group_size'] == 3]



set_group_size
1     44
4      1
5    328
dtype: int64

### Final check, by verifying with other fields

In [40]:
# data_desc_check = data
# data_desc_charms = data_desc_check[data_desc_check['Equip Slot'] == 'Charm']
# data_desc_armor = data_desc_check[~(data_desc_check['Equip Slot'] == 'Charm')]
# g_n_o = data_desc_armor.groupby(['Set Group', 'Order']).size().to_frame('group_and_order_size')
# data_desc_armor_gno = data_desc_armor.merge(g_n_o, on=['Set Group', 'Order'])
# data_desc_armor_gno

# sg = data_desc_armor_gno.groupby('Set Group').agg(set_group_size = ('Set Group', 'size'))
# sg.sort_values(by='set_group_size')
# sg.groupby('set_group_size').size()
# #sg[sg['Set Group'] == 1]

# data_desc_armor_both = data_desc_armor_gno.merge(sg, on='Set Group')
# data_desc_armor_both
# data_desc_armor_both[data_desc_armor_both['set_group_size'] == 1]
# #data_desc_armor_both[~(data_desc_armor_both['set_group_size'] == data_desc_armor_both['group_and_order_size'])]


Going to try to identify set names

In [70]:
import functools
import re

justArmor = data[~(data['Equip Slot'] == 'Charm')]
justCharms = data[(data['Equip Slot'] == 'Charm')]
set_group_sizes = justArmor.groupby(by='Set Group').size().to_frame('set_group_size')
justArmor = justArmor.merge(set_group_sizes, on='Set Group')

justSetArmor = justArmor[justArmor['set_group_size'] > 1]
singletonArmor = justArmor[justArmor['set_group_size'] == 1]
groupedData = justSetArmor.groupby(by='Set Group')

def removeIfExists(val, lst):
  if val in lst:
    lst.remove(val)
    return True
  else: return False

def listIntersection(list1, list2):
  outlist = [value for value in list1 if removeIfExists(value, list2)]
  return outlist

def findCommonWords(group):
  names = [name.split(' ') for name in group]
  commonWords = functools.reduce(listIntersection, names)
  return commonWords

setNameTuples = [(setGroupId, ' '.join(findCommonWords(group['Name'].values))) for setGroupId, group in groupedData]
setNames = pd.DataFrame(setNameTuples, columns=['Set Group', 'Set Name'])
setNames

justSetArmor = justSetArmor.merge(setNames, on='Set Group')
justSetArmor
justSetArmor['Set Name'].unique()
singletonArmor = singletonArmor.assign(SetName = 'N/A')
singletonArmor = singletonArmor.rename(columns={'SetName' : 'Set Name'})
justCharms = justCharms.copy()
justCharms.loc[:,'Set Name'] = justCharms['Name'].str.extract('(.*) Charm.*', expand=False)
notSetArmor = pd.concat([singletonArmor, justCharms])
notSetArmor = notSetArmor.drop(columns='set_group_size')
notSetArmor

,Name,Index,Order,Variant,Set (Layered) Id,Type,Equip Slot,Defense,Model Id 1,Model Id 2,...,Skill 1 Level,Skill 2,Skill 2 Level,Skill 3,Skill 3 Level,Gender,Set Group,Is_Permanent,Description,Set Name
14,Shamos Goggles,14,18,Low_Rank,161,Regular,Head,22,58,0,...,1,0: --------,0,0: --------,0,Unisex,168,False,These Shamos goggles have under- gone a specia...,N/A
31,Pulverizing Feather,31,0,Low_Rank,36,Regular,Head,32,37,0,...,3,0: --------,0,0: --------,0,Unisex,37,False,An earring left by the man they call the Demol...,N/A
46,Shamos Goggles α,46,57,Alpha,162,Regular,Head,40,58,0,...,1,100: Geologist,1,0: --------,0,Unisex,174,False,These Shamos goggles have under- gone a specia...,N/A
47,Shamos Goggles β,47,58,Beta_or_Gamma,163,Regular,Head,40,58,0,...,1,0: --------,0,0: --------,0,Unisex,175,False,These Shamos goggles have under- gone a specia...,N/A
76,Gastodon Horn α,76,85,Alpha,53,Regular,Head,48,15,0,...,1,6: Blast Resistance,1,0: --------,0,Unisex,1001,False,"The thick, stiff Gastodon hair used in this ar...",N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3457,Performer's Charm II,3457,141,Beta_or_Gamma,0,Regular,Charm,0,0,0,...,2,0: --------,0,0: --------,0,Unisex,378,False,A charm that enhances the Horn Maestro skill.,Performer's
3458,Survival Charm I,3458,1,Low_Rank,0,Regular,Charm,0,0,0,...,0,0: --------,0,0: --------,0,Unisex,379,False,A charm that enhances the Survival Expert skill.,Survival
3459,Survival Charm II,3459,2,Low_Rank,0,Regular,Charm,0,0,0,...,0,0: --------,0,0: --------,0,Unisex,380,False,A charm that enhances the Survival Expert skill.,Survival
3460,Survival Charm III,3460,3,Low_Rank,0,Regular,Charm,0,0,0,...,0,0: --------,0,0: --------,0,Unisex,381,False,A charm that enhances the Survival Expert skill.,Survival


In [71]:
from pathlib import Path
Path('.\\resources\\datamined_data_clean').mkdir(parents=True, exist_ok=True)
data.to_excel('.\\resources\\datamined_data_clean\\armor.xlsx')